In [1]:
using WebDataset
using Transducers

function stage(f::Function, rest...; csize=100, ntasks=1, verbose=false, kw...)
    outch = Channel(csize)
    return x -> begin
        if verbose; @show "starting", f, x, outch, rest; end
        Threads.@spawn f(x, outch, rest...; kw...)
        return outch
    end
end

function mkch(rest...)
    ch = Channel(length(rest))
    for s in rest
        put!(ch, s)
    end
    return ch
end

function show(ch)
    for item in ch
        sample = rstrip((string(item)*" "^100)[1:100], ' ')
        @show keys(item), sample
    end
end

┌ Info: Precompiling WebDataset [dfbb906c-54e0-4d57-86b9-79210db879da]
└ @ Base loading.jl:1317


show (generic function with 1 method)

In [5]:
iterator = sampleiterator("test.tar")
decoded = sampletransforms(iterator, default_decoders)
batched = samplebatching(decoded, 16)
count = 0
@time for batch in batched
    count += length(batch["__key__"])
end
count

 58.555050 seconds (3.58 M allocations: 17.678 GiB, 1.24% gc time, 0.28% compilation time)


8241

In [6]:
8241/58.5

140.87179487179486

In [7]:
ch = Channel(100)

Channel{Any}(100) (empty)

In [8]:
sampleiterator("test.tar") |> Take(10) |> ch

LoadError: MethodError: objects of type Channel{Any} are not callable

In [10]:
import Base: (|>)
function (|>)(source, ch::Channel)
    for sample in source
        put!(ch, sample)
    end
end

|> (generic function with 3 methods)

In [11]:
sampleiterator("test.tar") |> Take(10) |> ch

In [12]:
ch

Channel{Any}(100) (10 items available)